# Import Library

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import faiss
import pickle
import joblib
import ast

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import layers, models
from sentence_transformers import SentenceTransformer

# Prepare Data

In [5]:
df = pd.read_csv("../notebooks/anang/export/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,company_industry,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Akuntansi / Keuangan,Audit & Pajak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Konstruksi/Bangunan/Teknik,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",Supervisor,3,Middle,S1
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...",Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"Konsultasi (IT, Ilmu Pengetahuan, Teknis & Tek...","SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",Manajer,2,Middle,Unknown
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),Penuh Waktu,"Manufaktur,Manufaktur","Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Manufaktur/Produksi,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",Manajer,5,Senior,S1
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Komputer/Teknik Informatika (Perangkat Lunak),Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",Supervisor,2,Middle,S1
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Industri Berat/Mesin/Peralatan,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",Staff,1,Entry,S2


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2921 non-null   int64  
 1   job_title                2921 non-null   object 
 2   location                 2921 non-null   object 
 3   salary_currency          2921 non-null   object 
 4   career_level             2921 non-null   object 
 5   experience_level         2921 non-null   object 
 6   education_level          2921 non-null   object 
 7   employment_type          2921 non-null   object 
 8   job_function             2921 non-null   object 
 9   job_benefits             2921 non-null   object 
 10  company_process_time     2921 non-null   object 
 11  company_size             2921 non-null   object 
 12  company_industry         2921 non-null   object 
 13  job_description          2921 non-null   object 
 14  salary                  

In [7]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(len(unique_values_list))

63


In [8]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",
    "Sains,Pertanian": "Engineering",
    "Sains,Teknologi Makanan/Ahli Gizi": "Engineering",
    "Sains,Sains & Teknologi": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",
    "Pelayanan,Pengacara / Asisten Legal": "HR & Administration",
    "Pelayanan,Angkatan Bersenjata": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",
    "Layanan Kesehatan,Diagnosa/Lainnya": "Hospitality & Service",
    "Layanan Kesehatan,Farmasi": "Hospitality & Service",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Hospitality & Service",
    "Sains,Bioteknologi": "Hospitality & Service",
    "Sains,Biomedis": "Hospitality & Service",
    "Sains,Kimia": "Hospitality & Service",
    "Sains,Geologi/Geofisika": "Hospitality & Service",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

In [9]:
encoders_dict = {}

columns_to_encode = ['job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,job_function_group
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Akuntansi / Keuangan,Audit & Pajak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",Supervisor,3,Middle,S1,3
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...",Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",Manajer,2,Middle,Unknown,9
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),Penuh Waktu,"Manufaktur,Manufaktur","Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",Manajer,5,Senior,S1,7
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",Supervisor,2,Middle,S1,6
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",Staff,1,Entry,S2,6


In [10]:
y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 10
job_function_group
9    1038
6     598
3     283
7     243
4     217
2     163
0     131
5     110
8      74
1      64
Name: count, dtype: int64


In [11]:
RANDOM_STATE = 42
MAX_PER_CLASS = 600

dfs = []

for label, group in df.groupby("job_function_group"):
    if len(group) > MAX_PER_CLASS:
        group = group.sample(
            n=MAX_PER_CLASS,
            random_state=RANDOM_STATE
        )
    dfs.append(group)

df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2483 non-null   int64  
 1   job_title                2483 non-null   object 
 2   location                 2483 non-null   object 
 3   salary_currency          2483 non-null   object 
 4   career_level             2483 non-null   object 
 5   experience_level         2483 non-null   object 
 6   education_level          2483 non-null   object 
 7   employment_type          2483 non-null   object 
 8   job_function             2483 non-null   object 
 9   job_benefits             2483 non-null   object 
 10  company_process_time     2483 non-null   object 
 11  company_size             2483 non-null   object 
 12  company_industry         2483 non-null   object 
 13  job_description          2483 non-null   object 
 14  salary                  

In [12]:
df["role_text"] = df["job_description"] + " - " + df['experience_level'] + " - " + df["education_level"]

In [13]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,job_function_group,role_text
0,251,Desain Grafis,Jakarta Selatan,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Tip;Waktu regular, Senin - Jumat;Bisnis (conto...",...,5400000.0,tugas tanggung jawab membuat desain yang kreat...,{'text': 'tugas tanggung jawab membuat desain ...,"['adobe illustrator', 'adobe photoshop', 'prom...",Staff,2,Middle,S1,0,Tugas & Tanggung Jawab :Membuat Desain yang kr...
1,1025,Communication Officer (interviewed would be fu...,Sleman,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Seni/Media/Komunikasi,Hubungan Masyarakat","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,4200000.0,job description \twould be the main contact pe...,{'text': 'job description would be the main co...,"['microsoft office', 'microsoft team', 'planni...",Staff,2,Middle,D4,0,Job Description:1.\tWould be the main contact ...
2,1156,MANDARIN TRANSLATOR（中文翻译）,Jakarta Raya,IDR,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Lainnya,Jurnalis/Editor","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,9000000.0,kualifikasi 工作要求usia bisa berbahasa mandarin l...,{'text': 'kualifikasi 工作要求usia bisa berbahasa ...,"['microsoft word', 'microsoft excel', 'social ...",Fresh Graduate,1,Entry,S1,0,Kualifikasi : 工作要求Usia 20-38 Tahun年纪20-38岁之间Di...
3,1235,Video editor,Jakarta Pusat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",Tip;Asuransi kesehatan;Parkir;Kasual (contoh: ...,...,4500000.0,video editor design graphickualifikasi pendidi...,{'text': 'video editor design graphickualifika...,"['digital marketing', 'video design', 'video e...",Staff,1,Entry,S1,0,VIDEO EDITOR - DESIGN GRAPHICKualifikasi :Pend...
4,1634,Digital Marketing,Jakarta Pusat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",Tunjangan Pendidikan;Tip;Asuransi kesehatan;Bi...,...,5500000.0,tugas bertanggung jawab dengan konten kreatif ...,{'text': 'tugas bertanggung jawab dengan konte...,"['adobe photoshop', 'adobe illustrator', 'adob...",Staff,1,Entry,S1,0,Tugas :1. Bertanggung jawab dengan konten krea...


In [14]:
X_text = df["role_text"].astype(str)
y = df["job_function_group"].astype(int)

num_classes = y.nunique()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

data = (X_train, y_train), (X_test, y_test)

# Model for Predict Job Function

### N-gram vectors

In [16]:
NGRAM_RANGE = (1, 2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    kwargs = {
        'ngram_range': NGRAM_RANGE,
        'dtype': 'float32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,
        'min_df': MIN_DOCUMENT_FREQUENCY
    }
    vectorizer = TfidfVectorizer(**kwargs)

    X_train = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)

    selector = SelectKBest(f_classif, k=min(TOP_K, X_train.shape[1]))
    selector.fit(X_train, train_labels)
    X_train = selector.transform(X_train)
    X_val = selector.transform(X_val)

    X_train = X_train.toarray()
    X_val = X_val.toarray()
    
    return X_train, X_val, vectorizer, selector

In [17]:
# X_train, X_test = ngram_vectorize(X_train, y_train, X_test)

### Last Layer

In [18]:
def _get_last_layer_units_and_activation(num_classes):
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

### Build n-gram model

In [19]:
def mlp_model(num_layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dropout(rate=dropout_rate))

    for _ in range(num_layers-1):
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate=dropout_rate))

    model.add(layers.Dense(units=op_units, activation=op_activation))
    return model

### Train Your Model

In [20]:
(train_texts, train_labels), (val_texts, val_labels) = data
X_train, X_val, vectorizer, selector = ngram_vectorize(train_texts, train_labels, val_texts)

c:\Users\ACER\Documents\Dokumen\Data Science Dicoding\Project Capstone\Project\Career-Path-Navigator\navigator-env\Lib\site-packages\sklearn\feature_extraction\text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  warnings.warn(


In [21]:
joblib.dump(
    {"vectorizer": vectorizer, "selector": selector},
    "text_preprocessor.joblib"
)

['text_preprocessor.joblib']

In [22]:
def train_ngram_model(
    data,
    learning_rate=1e-3,
    epochs=1000,
    batch_size=128,
    num_layers=2,
    units=64,
    dropout_rate=0.2
):

    model = mlp_model(
        num_layers=num_layers,
        units=units,
        dropout_rate=dropout_rate,
        input_shape=X_train.shape[1:],
        num_classes=num_classes
    )

    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2
    )]

    history = model.fit(
        X_train,
        train_labels,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, val_labels),
        verbose=2,
        batch_size=batch_size
    )

    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    model.save('mlp_model.keras')
    return history['val_acc'][-1], history['val_loss'][-1]

In [23]:
train_ngram_model(data, learning_rate=1e-3, epochs=1000, batch_size=128, num_layers=2, units=64, dropout_rate=0.2)

Epoch 1/1000
16/16 - 1s - 53ms/step - acc: 0.4466 - loss: 2.2385 - val_acc: 0.5252 - val_loss: 2.1374
Epoch 2/1000
16/16 - 0s - 14ms/step - acc: 0.5926 - loss: 2.0172 - val_acc: 0.5594 - val_loss: 1.9143
Epoch 3/1000
16/16 - 0s - 12ms/step - acc: 0.6511 - loss: 1.7511 - val_acc: 0.5936 - val_loss: 1.6852
Epoch 4/1000
16/16 - 0s - 12ms/step - acc: 0.6868 - loss: 1.4978 - val_acc: 0.6338 - val_loss: 1.4873
Epoch 5/1000
16/16 - 0s - 14ms/step - acc: 0.7367 - loss: 1.2713 - val_acc: 0.6761 - val_loss: 1.3242
Epoch 6/1000
16/16 - 0s - 13ms/step - acc: 0.7840 - loss: 1.0778 - val_acc: 0.6982 - val_loss: 1.1882
Epoch 7/1000
16/16 - 0s - 13ms/step - acc: 0.8348 - loss: 0.9086 - val_acc: 0.7465 - val_loss: 1.0749
Epoch 8/1000
16/16 - 0s - 11ms/step - acc: 0.8701 - loss: 0.7768 - val_acc: 0.7686 - val_loss: 0.9854
Epoch 9/1000
16/16 - 0s - 14ms/step - acc: 0.8837 - loss: 0.6670 - val_acc: 0.7726 - val_loss: 0.9117
Epoch 10/1000
16/16 - 0s - 13ms/step - acc: 0.9074 - loss: 0.5786 - val_acc: 0.788

(0.8229376077651978, 0.5765637159347534)

### Inferensi

In [24]:
model = models.load_model('mlp_model.keras')

def predict_job_function(texts, vectorizer, selector, model):
    X = vectorizer.transform(texts)
    X = selector.transform(X).toarray()
    probs = model.predict(X)
    labels = probs.argmax(axis=1)
    return labels, probs

In [25]:
persona_0_creative_media = ["""Saya memiliki pengalaman panjang di bidang kreatif dan media, dengan fokus utama pada produksi konten visual dan pengelolaan komunikasi digital. 
Selama beberapa tahun terakhir, saya terbiasa bekerja dengan berbagai format konten seperti desain grafis, video pendek, fotografi, serta penulisan 
konten untuk media sosial dan platform digital lainnya. Saya memahami bagaimana sebuah pesan dapat disampaikan secara efektif melalui visual dan 
narasi yang kuat, sehingga audiens dapat terhubung secara emosional dengan brand atau produk yang ditampilkan. Dalam pekerjaan sehari-hari, saya 
sering terlibat dalam proses brainstorming ide kreatif bersama tim, mulai dari tahap konsep hingga eksekusi akhir. Saya terbiasa mengembangkan 
konsep kampanye, menentukan tone komunikasi, serta memastikan konsistensi identitas visual di berbagai kanal. Selain itu, saya juga terbiasa 
bekerja dengan deadline yang ketat dan melakukan revisi berdasarkan masukan dari stakeholder atau klien. Saya memiliki ketertarikan yang besar 
terhadap tren media digital, perkembangan platform sosial, dan perubahan perilaku audiens. Hal ini mendorong saya untuk terus belajar dan 
menyesuaikan gaya konten agar tetap relevan. Saya terbiasa menggunakan berbagai tools desain dan editing, serta memahami dasar analisis performa 
konten untuk melihat efektivitas kampanye yang dijalankan. Bagi saya, dunia kreatif bukan hanya soal estetika, tetapi juga tentang bagaimana sebuah
ide dapat dikomunikasikan dengan jelas dan berdampak. Saya menikmati proses eksplorasi ide, eksperimen visual, dan kolaborasi lintas fungsi dengan 
tim marketing maupun komunikasi. Saya tertarik untuk terus berkembang di bidang creative dan media, khususnya dalam peran yang memungkinkan saya 
berkontribusi pada pengembangan brand dan storytelling yang kuat.
"""]

persona_1_education = ["""
Saya memiliki latar belakang dan pengalaman panjang di bidang pendidikan, baik dalam konteks pengajaran langsung maupun pengembangan materi 
pembelajaran. Saya terbiasa berinteraksi dengan peserta didik dari berbagai latar belakang, memahami kebutuhan belajar mereka, serta menyesuaikan 
metode pengajaran agar materi dapat dipahami dengan lebih efektif. Saya percaya bahwa proses belajar tidak hanya tentang penyampaian materi, tetapi 
juga tentang membangun pemahaman, motivasi, dan rasa ingin tahu. Dalam peran saya sebagai pendidik, saya sering terlibat dalam penyusunan silabus, 
perencanaan pembelajaran, serta evaluasi hasil belajar. Saya terbiasa menggunakan berbagai pendekatan, baik konvensional maupun berbasis teknologi, 
untuk mendukung proses belajar mengajar. Saya juga memiliki pengalaman dalam melakukan pendampingan dan bimbingan kepada peserta didik yang 
membutuhkan perhatian khusus. Saya memiliki ketertarikan pada pengembangan sistem pendidikan, inovasi metode pembelajaran, serta pemanfaatan 
teknologi untuk meningkatkan kualitas pendidikan. Saya percaya bahwa pendidikan memiliki peran penting dalam membentuk karakter dan kompetensi 
seseorang. Oleh karena itu, saya selalu berusaha menciptakan lingkungan belajar yang kondusif, inklusif, dan mendorong partisipasi aktif. Selain 
mengajar, saya juga terbiasa bekerja secara administratif di lingkungan pendidikan, seperti pengelolaan data akademik dan koordinasi kegiatan 
pembelajaran. Saya memiliki kemampuan komunikasi yang baik, sabar, dan terorganisir. Saya tertarik untuk terus berkontribusi di dunia pendidikan,
baik sebagai pengajar, pengembang kurikulum, maupun peran lain yang mendukung proses belajar.
"""]

persona_2_engineering = ["""
Saya memiliki latar belakang di bidang engineering dan pengalaman bekerja dalam lingkungan teknis yang menuntut ketelitian serta pemahaman sistem
yang kuat. Saya terbiasa menganalisis permasalahan teknis, merancang solusi, serta memastikan implementasi berjalan sesuai dengan spesifikasi dan 
standar yang telah ditetapkan. Dalam pekerjaan saya, aspek keselamatan, efisiensi, dan keandalan selalu menjadi prioritas utama. Saya sering bekerja 
dengan dokumentasi teknis, perhitungan, serta koordinasi dengan berbagai pihak seperti tim operasional dan manajemen proyek. Saya memahami bahwa 
sebuah solusi engineering tidak hanya harus bekerja secara teknis, tetapi juga harus dapat diterapkan secara praktis dan berkelanjutan. Oleh karena
itu, saya terbiasa mempertimbangkan berbagai faktor seperti biaya, waktu, dan risiko dalam setiap keputusan teknis. Saya menikmati proses pemecahan
masalah dan tantangan teknis yang kompleks. Saya terbiasa bekerja secara sistematis dan terstruktur, serta mampu bekerja baik secara mandiri maupun 
dalam tim. Saya juga memiliki ketertarikan untuk terus mempelajari teknologi dan metode baru yang dapat meningkatkan kualitas solusi engineering 
yang saya kerjakan. Bagi saya, bidang engineering adalah tentang mengubah konsep dan teori menjadi solusi nyata yang dapat memberikan manfaat. 
Saya tertarik untuk terus berkembang dalam peran engineering yang memungkinkan saya berkontribusi pada pengembangan sistem, infrastruktur, atau
produk yang andal dan efisien.
"""]

persona_3_finance_accounting = ["""
Saya memiliki pengalaman panjang di bidang keuangan dan akuntansi, dengan fokus pada pengelolaan data keuangan yang akurat dan transparan. 
Saya terbiasa menangani pencatatan transaksi, penyusunan laporan keuangan, serta analisis anggaran. Ketelitian dan konsistensi adalah hal yang 
sangat saya jaga dalam bekerja dengan angka dan data finansial. Dalam keseharian, saya sering bekerja dengan laporan keuangan bulanan dan tahunan,
melakukan rekonsiliasi, serta memastikan kepatuhan terhadap standar akuntansi. Saya memahami pentingnya data keuangan sebagai dasar pengambilan 
keputusan bisnis, sehingga saya selalu berusaha menyajikan informasi yang jelas dan dapat dipercaya. Saya juga terbiasa bekerja dengan berbagai 
pihak, termasuk tim internal dan auditor, untuk memastikan proses keuangan berjalan dengan baik. Saya memiliki kemampuan analisis yang kuat dan 
mampu mengidentifikasi potensi risiko maupun peluang dari sisi keuangan. Selain itu, saya terbiasa menggunakan tools seperti spreadsheet dan sistem
akuntansi untuk mendukung pekerjaan. Saya tertarik untuk terus berkembang di bidang finance dan accounting, khususnya dalam peran yang
memungkinkan saya berkontribusi pada stabilitas dan pertumbuhan organisasi melalui pengelolaan keuangan yang baik.
"""]

persona_4_hr_administration = ["""
Saya memiliki pengalaman di bidang human resources dan administrasi, dengan fokus pada pengelolaan sumber daya manusia dan proses administratif 
perusahaan. Saya terbiasa menangani data karyawan, administrasi kepegawaian, serta mendukung proses rekrutmen dan onboarding. Saya memahami
pentingnya ketertiban administrasi dalam mendukung operasional organisasi. Dalam peran saya, saya sering menjadi penghubung antara karyawan dan 
manajemen, memastikan informasi tersampaikan dengan jelas dan tepat. Saya memiliki kemampuan komunikasi yang baik dan terbiasa menangani dokumen 
serta data dengan tingkat kerahasiaan yang tinggi. Saya juga terbiasa bekerja dengan prosedur dan kebijakan perusahaan. Saya tertarik pada 
pengembangan sistem HR yang efisien dan berorientasi pada kesejahteraan karyawan. Saya percaya bahwa pengelolaan sumber daya manusia yang baik 
dapat menciptakan lingkungan kerja yang produktif dan harmonis. Oleh karena itu, saya selalu berusaha bekerja secara rapi, terstruktur, dan
profesional dalam setiap tugas administrasi yang saya tangani.
"""]

persona_5_hospitality_service = ["""
Saya memiliki pengalaman panjang di bidang hospitality dan layanan pelanggan, dengan fokus pada pemberian pelayanan yang ramah dan profesional. 
Saya terbiasa berinteraksi langsung dengan pelanggan, memahami kebutuhan mereka, serta memastikan pengalaman yang positif selama menggunakan 
layanan. Saya memahami bahwa kepuasan pelanggan merupakan faktor utama dalam industri jasa. Saya sering bekerja dalam lingkungan yang dinamis dan 
menuntut kecepatan serta ketepatan. Saya terbiasa menangani keluhan pelanggan dengan sikap tenang dan solutif. Selain itu, saya juga memahami 
pentingnya kerja sama tim dalam memastikan kelancaran operasional layanan. Saya memiliki kemampuan komunikasi interpersonal yang baik dan mampu 
bekerja dalam sistem shift. Saya menikmati pekerjaan yang melibatkan interaksi dengan banyak orang dan memberikan solusi secara langsung. 
Saya tertarik untuk terus berkembang di bidang hospitality dan service, khususnya dalam peran yang menekankan kualitas layanan dan kepuasan
pelanggan.
"""]

persona_6_it_software = ["""
Saya memiliki latar belakang dan pengalaman di bidang teknologi informasi dan pengembangan perangkat lunak. Saya terbiasa bekerja dengan sistem
berbasis software, memahami alur pengembangan aplikasi, serta memecahkan masalah teknis menggunakan pendekatan logis. Saya memiliki ketertarikan
besar terhadap teknologi dan bagaimana teknologi dapat digunakan untuk meningkatkan efisiensi kerja. Saya sering bekerja dengan data, aplikasi,
dan sistem backend, serta memahami konsep database dan integrasi sistem. Saya terbiasa bekerja secara terstruktur dan mengikuti proses pengembangan 
yang sistematis. Selain itu, saya juga mampu bekerja dalam tim dan berkomunikasi dengan pihak non-teknis untuk menjelaskan solusi yang dikembangkan.
Saya tertarik untuk terus belajar dan mengikuti perkembangan teknologi terbaru. Bagi saya, bidang IT dan software adalah ruang yang terus 
berkembang dan menawarkan tantangan intelektual yang menarik. Saya ingin terus berkontribusi dalam pengembangan solusi digital yang bermanfaat
dan scalable.
"""]

persona_7_manufacturing_operations = ["""
Saya memiliki pengalaman di bidang manufaktur dan operasional, dengan fokus pada pengawasan proses produksi dan efisiensi kerja. 
Saya terbiasa bekerja di lingkungan yang terstruktur dengan prosedur yang jelas. Saya memahami pentingnya kualitas produk, keselamatan kerja, dan 
kelancaran operasional dalam proses manufaktur. Saya sering terlibat dalam pemantauan proses produksi, koordinasi dengan tim, serta memastikan
target operasional tercapai. Saya terbiasa bekerja dengan jadwal dan target yang ketat, serta memahami pentingnya kerja sama tim dalam 
lingkungan produksi. Saya tertarik pada peningkatan efisiensi proses dan optimalisasi operasional. Saya percaya bahwa perbaikan kecil yang 
konsisten dapat memberikan dampak besar pada produktivitas. Saya ingin terus berkembang di bidang manufacturing dan operations dengan fokus 
pada proses yang efektif dan berkelanjutan.
"""]

persona_8_other = ["""
Saya memiliki latar belakang pekerjaan yang bersifat umum dan lintas fungsi. Saya terbiasa menangani berbagai tugas yang beragam, mulai dari
administrasi hingga dukungan operasional. Saya fleksibel dan mudah beradaptasi dengan lingkungan kerja yang berubah. Saya memiliki kemampuan
belajar yang cepat dan siap mendukung berbagai kebutuhan organisasi. Saya terbiasa bekerja dengan berbagai tim dan memahami pentingnya komunikasi 
yang jelas. Saya tertarik pada peran yang memungkinkan saya berkontribusi secara luas dan mendukung kelancaran operasional perusahaan.
"""]

persona_9_sales_marketing = ["""
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.
"""]

In [26]:
# persona_0_creative_media = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan besar di bidang kreatif dan media. Selama masa perkuliahan, saya aktif 
# mempelajari dasar-dasar desain visual, produksi konten digital, dan komunikasi kreatif. Saya terbiasa mengerjakan tugas kuliah yang berkaitan 
# dengan pembuatan konten media sosial, desain poster, presentasi visual, serta penulisan konten sederhana. Saya memiliki minat kuat pada storytelling 
# visual dan bagaimana sebuah pesan dapat disampaikan secara menarik kepada audiens. Saya terbiasa bekerja dengan tools desain dasar dan senang 
# mengikuti tren media digital serta perkembangan platform sosial. Sebagai fresh graduate, saya terbuka untuk belajar, menerima masukan, dan 
# mengembangkan kemampuan kreatif saya melalui pengalaman kerja di industri kreatif dan media.
# """]

# persona_1_education = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang pendidikan yang memiliki minat besar di dunia pendidikan dan pembelajaran. 
# Selama masa studi, saya terbiasa mengikuti kegiatan akademik, diskusi kelompok, serta praktik penyusunan materi pembelajaran. Saya memiliki 
# ketertarikan pada proses belajar mengajar dan bagaimana metode pembelajaran dapat disesuaikan dengan kebutuhan peserta didik. Saya juga memiliki 
# pengalaman membantu teman sebaya dalam kegiatan belajar dan aktif dalam organisasi kampus yang berhubungan dengan pendidikan. Saya percaya bahwa 
# pendidikan memiliki peran penting dalam pengembangan individu. Sebagai fresh graduate, saya ingin terus belajar dan berkontribusi dalam bidang 
# pendidikan, baik sebagai pengajar pemula, asisten pendidikan, maupun peran pendukung lainnya.
# """]

# persona_2_engineering = ["""
# Saya merupakan mahasiswa fresh graduate di bidang engineering dengan pemahaman dasar yang kuat terhadap konsep teknis dan pemecahan masalah. 
# Selama perkuliahan, saya terbiasa mengerjakan tugas dan proyek yang menuntut analisis, perancangan, serta penerapan konsep engineering secara 
# sistematis. Saya memiliki minat pada bagaimana teori dapat diterapkan menjadi solusi nyata. Saya terbiasa bekerja dengan perhitungan, simulasi, 
# dan dokumentasi teknis sederhana. Saya juga pernah terlibat dalam proyek kelompok yang melatih kerja sama tim dan komunikasi teknis. Sebagai fresh 
# graduate, saya siap belajar dari pengalaman lapangan dan mengembangkan kemampuan engineering saya secara bertahap.
# """]

# persona_3_finance_accounting = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang keuangan dan akuntansi. Selama masa kuliah, saya mempelajari dasar-dasar pencatatan 
# keuangan, penyusunan laporan keuangan, serta analisis sederhana terhadap data finansial. Saya terbiasa bekerja dengan angka dan data, serta 
# memahami pentingnya ketelitian dalam pengelolaan keuangan. Saya memiliki pengalaman mengerjakan studi kasus dan tugas kelompok yang berkaitan 
# dengan laporan keuangan dan anggaran. Saya tertarik untuk mengembangkan karier di bidang finance dan accounting dan siap belajar langsung dari 
# praktik kerja di dunia profesional.
# """]

# persona_4_hr_administration = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan pada bidang human resources dan administrasi. Selama perkuliahan, saya terbiasa 
# mengelola dokumen, data akademik, serta terlibat dalam kegiatan organisasi yang melibatkan administrasi dan koordinasi. Saya memiliki kemampuan 
# komunikasi yang baik dan terbiasa bekerja secara rapi dan terstruktur. Saya tertarik pada proses pengelolaan karyawan, rekrutmen, serta administrasi 
# perusahaan. Sebagai fresh graduate, saya siap belajar mengenai sistem HR dan administrasi secara profesional serta berkontribusi mendukung 
# operasional organisasi.
# """]

# persona_5_hospitality_service = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki minat besar di bidang hospitality dan layanan. Selama masa kuliah, saya terbiasa berinteraksi 
# dengan banyak orang melalui kegiatan organisasi, kepanitiaan, maupun kerja part-time. Saya memahami pentingnya sikap ramah, komunikasi yang baik, 
# dan pelayanan yang profesional. Saya terbiasa bekerja dalam situasi yang dinamis dan melibatkan kerja tim. Saya tertarik untuk mengembangkan karier 
# di bidang hospitality dan service dengan fokus pada kepuasan pelanggan dan kualitas layanan.
# """]

# persona_6_it_software = ["""
# Saya merupakan mahasiswa fresh graduate di bidang IT dan software dengan pemahaman dasar tentang pemrograman dan sistem informasi. Selama kuliah, 
# saya terbiasa mengerjakan tugas dan proyek yang berkaitan dengan pengembangan aplikasi sederhana, pengolahan data, dan pemecahan masalah teknis. 
# Saya memiliki ketertarikan besar pada teknologi dan senang mempelajari tools serta bahasa pemrograman baru. Saya terbiasa bekerja secara logis dan 
# terstruktur, serta mampu belajar secara mandiri. Sebagai fresh graduate, saya siap mengembangkan kemampuan teknis saya melalui pengalaman kerja 
# dan bimbingan dari tim profesional.
# """]

# persona_7_manufacturing_operations = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan pada bidang manufaktur dan operasional. Selama perkuliahan, saya mempelajari 
# dasar-dasar proses produksi, manajemen operasional, dan efisiensi kerja. Saya terbiasa mengikuti prosedur dan memahami pentingnya kualitas serta 
# keselamatan kerja. Saya memiliki pengalaman kerja kelompok yang melatih kedisiplinan dan koordinasi tim. Sebagai fresh graduate, saya ingin belajar 
# langsung mengenai proses operasional di lapangan dan berkontribusi dalam peningkatan efisiensi kerja.
# """]

# persona_8_other = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang umum dan fleksibel. Selama masa kuliah, saya terbiasa mengerjakan berbagai jenis 
# tugas akademik dan terlibat dalam kegiatan organisasi kampus. Saya mudah beradaptasi, memiliki kemampuan belajar yang cepat, dan siap membantu 
# berbagai kebutuhan tim. Saya tertarik pada peran entry-level yang memberikan kesempatan untuk belajar lintas fungsi dan mengembangkan pengalaman 
# kerja secara menyeluruh.
# """]

# persona_9_sales_marketing = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki minat di bidang sales dan marketing. Selama masa kuliah, saya terbiasa melakukan presentasi, 
# diskusi, dan kerja kelompok yang melatih kemampuan komunikasi dan persuasi. Saya tertarik pada strategi pemasaran, perilaku konsumen, dan proses 
# penjualan. Saya juga pernah terlibat dalam kegiatan organisasi atau proyek kampus yang berkaitan dengan promosi dan event. Sebagai fresh graduate, 
# saya siap belajar langsung di dunia kerja dan mengembangkan kemampuan sales serta marketing secara profesional.
# """]

In [27]:
persona_random = ["""
Saya adalah seekor kucing dengan kemampuan untuk mencuri makanan, tidur siang panjang, lapar setiap 5 menit, beraktivitas setiap hari, nyakar jok motor
, bercita-cita untuk bisa menjadi sales. Saya menganggap manusia adalah babu yang perlu melayani saya, menggaruk leher saya ketika gatal. Visi saya adalah
untuk menguasasi negara, menanam sawit di kalimantan dan papua."""]

labels, probs = predict_job_function(persona_9_sales_marketing, vectorizer, selector, model)
print(labels)
print(probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[9]
[[0.00271552 0.00166376 0.0013004  0.00136611 0.00250376 0.00152264
  0.00217681 0.00196523 0.00334845 0.9814373 ]]


# Model untuk Get Recommendation based on Semantic Matching

### Embedding

In [28]:
EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts, batch_size=64):
    return embed_model.encode(texts, show_progress_bar=True, batch_size=batch_size, convert_to_numpy=True)

role_texts = df["role_text"].tolist()
role_embeddings = embed_texts(role_texts)
print("role_embeddings", role_embeddings.shape)

skills_texts = df["skills"].tolist()
skill_embeddings = embed_texts(skills_texts)
print("skill_embeddings", skill_embeddings.shape)

c:\Users\ACER\Documents\Dokumen\Data Science Dicoding\Project Capstone\Project\Career-Path-Navigator\navigator-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mess

role_embeddings (2483, 384)


Batches: 100%|██████████| 39/39 [00:36<00:00,  1.07it/s]

skill_embeddings (2483, 384)


### Indexing

In [29]:
d = role_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(role_embeddings)
print("index ntotal:", index.ntotal)

faiss.write_index(index, "faiss_role_index.idx")
np.save("role_embeddings.npy", role_embeddings)
np.save("skill_embeddings.npy", skill_embeddings)

index ntotal: 2483


### Create Metadata

In [30]:
metadata = df[[
    "id", "job_title", "job_function", "skills", "job_description"
]].to_dict(orient="records")

In [31]:
with open("job_metadata.bin", "wb") as f:
    pickle.dump(metadata, f)

### Search

In [32]:
def search(query_embedding, k=10):
        D, I = index.search(
            query_embedding, k
        )

        results = []
        for dist, idx in zip(D[0], I[0]):
            job = metadata[idx].copy()
            job["distance"] = float(dist)
            job["similarity"] = float(1 / (1 + dist))  # optional
            results.append(job)

        return results

### Get Top Roles by Text

In [33]:
def get_top_roles_by_text(user_text, k=3):
    emb = embed_texts([user_text])
    results = search(query_embedding=emb, k=k)
    return results

In [34]:
rekomendasi = get_top_roles_by_text(persona_9_sales_marketing[0], k=5)

Batches: 100%|██████████| 1/1 [00:00<00:00, 26.80it/s]


In [35]:
rekomendasi

[{'id': 26603,
  'job_title': 'AREA SALES MANAGER, KALIMANTAN',
  'job_function': 'Penjualan / Pemasaran,Telesales/Telemarketing',
  'skills': "['sales', 'survey', 'c', 'bahasa']",
  'job_description': 'TANGGUNG JAWAB\xa0Merencanakan dan mengembangan strategi penjualan dan pemasaran yang inovatif dan up to date untuk perkiraan penjualan\xa0Mengembangan penjualan melalui distributor baru\xa0Menjalin hubungan baik dan mengatasi hambatan penjualan dengan distributorMenetapkan target penjualanBertanggung jawab atas pencapaian target penjualan di areanyaMemastikan proses kerja tim sales yang efektif dan efisienMelakukan pengawasan dan evaluasi dalam kompetensi, kinerja, dan sikap tim sales\xa0Memberikan pengarahan dan pengembangan untuk meningkatkan keterampilan tim salesTUGAS UTAMA\xa0Mengkoordinir tim sales agar dapat meningkatkan penjualan sesuai target\xa0Mengelola, melatih dan mendorong tim sales untuk meningkatkan penjualan di areanya\xa0Monitoring kinerja tim sales yang berada di are

# Skill Gap Module

**Buat Model TF-IDF untuk Skill Gap Module**

TfidfVectorizer dilatih (fit) khusus pada data skills.

In [44]:
# Siapkan TF-IDF
# Kita buat vectorizer khusus untuk kolom 'skills'
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)

# Pastikan kolom skills dibaca sebagai string
skills_data = df['skills'].astype(str).fillna('')
tfidf.fit(skills_data)

# Siapkan Global Importance
# Hitung rata-rata bobot TF-IDF 
tfidf_matrix = tfidf.transform(skills_data)
global_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
feature_names = tfidf.get_feature_names_out()

# Buat dictionary: { 'nama_skill': skor_kepentingan }
global_importance_dict = dict(zip(feature_names, global_scores))

**Skill Gap Module**

Dalam modul rekomendasi skill (Skill Gap Module), sistem tidak menggunakan rata-rata aritmatika biasa untuk memberikan skor rekomendasi. Sebaliknya, kami menerapkan Rerata Harmonis (Harmonic Mean), sebuah pendekatan yang sama dengan perhitungan F1-Score dalam evaluasi klasifikasi.

Skor akhir ($S$) untuk setiap kandidat skill dihitung berdasarkan dua komponen:

1. Relevance ($R$): Seberapa besar "kekosongan" (gap) skill ini pada target pekerjaan yang dituju user? (Dihitung dari selisih vektor TF-IDF).
2. Importance ($I$): Seberapa vital skill ini secara global menurut model Random Forest? (Dihitung dari Feature Importance).

Harmonic Mean Logic

$$Score = \frac{2 \times (Relevance \times Importance)}{Relevance + Importance}$$

Dengan menggunakan Harmonic Mean, sistem menjamin bahwa skill yang direkomendasikan adalah skill yang benar-benar "High Impact" secara global, namun tetap "Contextually Accurate" dengan tujuan karir user.

In [45]:
class SkillGapModule:
    def __init__(self, vectorizer, df, global_imp_dict):
        self.vectorizer = vectorizer
        self.feature_names = vectorizer.get_feature_names_out()
        self.global_imp = global_imp_dict
        self.df = df 
        
    def _clean_skill_string(self, skill_str):
        """
        Fungsi untuk membersihkan string skill
        """
        try:
            # Jika bentuknya string list python
            if skill_str.startswith("[") and skill_str.endswith("]"):
                skill_list = ast.literal_eval(skill_str)
                return " ".join(skill_list)
            return skill_str
        except:
            return skill_str

    def get_target_vector_from_recs(self, recommendation_list):
        """
        Membuat vektor target berdasarkan RATA-RATA skill dari top rekomendasi.
        """
        if not recommendation_list:
            return None
        
        # Ambil skill dari setiap job di rekomendasi
        cleaned_skills = []
        for job in recommendation_list:
            raw_skill = str(job.get('skills', ''))
            cleaned_text = self._clean_skill_string(raw_skill)
            cleaned_skills.append(cleaned_text)
            
        # Buat Matrix TF-IDF untuk job-job rekomendasi ini
        target_vec_matrix = self.vectorizer.transform(cleaned_skills)
        
        # Hitung Centroid (Rata-rata vektor)
        # Ini merepresentasikan "Skill Ideal" untuk cluster rekomendasi ini
        composite_vector = np.asarray(target_vec_matrix.mean(axis=0)).flatten()
        
        return composite_vector

    def analyze(self, user_text, recommendation_list, top_n=10):
        """
        user_text: Text input dari user (persona)
        recommendation_list: List dictionary hasil search() atau get_top_roles_by_text()
        """
        # Dapatkan Vektor Target dari Rekomendasi
        target_vec = self.get_target_vector_from_recs(recommendation_list)
        
        if target_vec is None:
            return []
        
        # Hitung Vektor User (Skill yang user punya)
        user_vec = self.vectorizer.transform([user_text]).toarray().flatten()
        
        # Hitung Gap (Target - User) -> Ambil yang positif saja (Missing Skills)
        gap_vec = np.maximum(0, target_vec - user_vec)
        
        # Threshold filter noise (agar skill dengan bobot sangat kecil tidak muncul)
        gap_indices = np.where(gap_vec > 0.005)[0]
        
        if len(gap_indices) == 0:
            return [] # Tidak ada gap signifikan

        # Scoring Kandidat Skill
        candidates = []
        for idx in gap_indices:
            skill_name = self.feature_names[idx]
            
            # RELEVANCE: Seberapa besar gap skill ini di job rekomendasi?
            relevance = gap_vec[idx]
            
            # IMPORTANCE: Seberapa penting skill ini secara global (Model MLP)?
            importance = self.global_imp.get(skill_name, 0)
            
            candidates.append({
                'skill': skill_name,
                'relevance': relevance,
                'importance': importance
            })
            
        if not candidates:
            return []
            
        # Normalisasi & Harmonic Mean (F1-like Score)
        df_cand = pd.DataFrame(candidates)
        scaler = MinMaxScaler()
        
        if len(df_cand) > 1:
            scaled_vals = scaler.fit_transform(df_cand[['relevance', 'importance']])
            df_cand['rel_norm'] = scaled_vals[:, 0]
            df_cand['imp_norm'] = scaled_vals[:, 1]
        else:
            df_cand['rel_norm'] = 1.0
            df_cand['imp_norm'] = 1.0
            
        epsilon = 1e-5
        # Rumus Harmonic Mean: Mengutamakan skill yang Relevan DAN Penting
        df_cand['final_score'] = 2 * (df_cand['rel_norm'] * df_cand['imp_norm']) / \
                                     (df_cand['rel_norm'] + df_cand['imp_norm'] + epsilon)
        
        # Ambil Top N Score Tertinggi
        results = df_cand.sort_values('final_score', ascending=False).head(top_n)
        
        return results[['skill', 'final_score']].to_dict('records')

# Inisiasi Modul
gap_analyzer = SkillGapModule(tfidf, df, global_importance_dict)

Menggabungkan hasil pencarian embedding dengan analisis gap

In [46]:
# Ambil Input User
user_persona = persona_9_sales_marketing[0]
print(f"User Input: {user_persona[:100]}...")

# Dapatkan Rekomendasi Job (Semantic Search)
rekomendasi = get_top_roles_by_text(user_persona, k=5)

# Tampilkan Judul Job yang direkomendasikan
print("\nRekomendasi Pekerjaan:")
for job in rekomendasi:
    print(f"- {job['job_title']} (Similarity: {job['similarity']:.3f})")

# Analisis Skill Gap
# Bandingkan User vs Rata-rata Skill dari 5 Job Rekomendasi tersebut
skill_gaps = gap_analyzer.analyze(user_persona, rekomendasi, top_n=10)

print("\n--- SKILL GAP ANALYSIS ---")
if not skill_gaps:
    print("Selamat! Skill kamu sangat cocok, tidak ada gap signifikan.")
else:
    print("Skill yang perlu dipelajari untuk pekerjaan di atas:")
    for i, gap in enumerate(skill_gaps, 1):
        print(f"{i}. {gap['skill']} (Score: {gap['final_score']:.4f})")

User Input: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pe...


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.25it/s]


Rekomendasi Pekerjaan:
- AREA SALES MANAGER, KALIMANTAN (Similarity: 0.189)
- AREA SALES MANAGER, KALIMANTAN (Similarity: 0.189)
- Sales Marketing (Similarity: 0.171)
- Marketing Cosmetics & SkinCare Manager (Similarity: 0.161)
- Marketing & Development Staff (Similarity: 0.161)

--- SKILL GAP ANALYSIS ---
Skill yang perlu dipelajari untuk pekerjaan di atas:
1. bahasa (Score: 0.3978)
2. business (Score: 0.2703)
3. survey (Score: 0.2528)
4. oriented (Score: 0.1665)
5. target oriented (Score: 0.1502)
6. customer (Score: 0.1477)
7. communication (Score: 0.1454)
8. product (Score: 0.1438)
9. management (Score: 0.1436)
10. ide (Score: 0.1378)


**Buat Opsi Skill Gap Direct**

Skill Gap Module sebelumnya menghimpun skill yang dibutuhkan secara global, untuk opsi kedua kali ini skill gap module dibuat secara sederhana sesuai dari himpunan skill yang berasal dari rekomendasi job.

In [47]:
class DirectSkillGapModule:
    def __init__(self, vectorizer):
        # Kita hanya butuh vectorizer (TF-IDF) untuk menerjemahkan teks ke angka
        self.vectorizer = vectorizer
        self.feature_names = vectorizer.get_feature_names_out()
        
    def _clean_skill_string(self, skill_str):
        """Membersihkan format string list"""
        try:
            if isinstance(skill_str, str) and skill_str.strip().startswith("["):
                skill_list = ast.literal_eval(skill_str)
                return " ".join(skill_list)
            return str(skill_str)
        except:
            return str(skill_str)

    def get_target_vector_from_recs(self, recommendation_list):
        """
        Membuat 'Profil Skill Ideal' berdasarkan rata-rata skill 
        dari pekerjaan yang direkomendasikan saja.
        """
        if not recommendation_list:
            return None
        
        cleaned_skills = []
        for job in recommendation_list:
            raw_skill = job.get('skills', '')
            cleaned_text = self._clean_skill_string(raw_skill)
            cleaned_skills.append(cleaned_text)
            
        if not cleaned_skills:
            return None

        # Buat centroid (rata-rata vector)
        # Skill yang muncul di BANYAK rekomendasi akan memiliki nilai tinggi disini.
        target_vec_matrix = self.vectorizer.transform(cleaned_skills)
        composite_vector = np.asarray(target_vec_matrix.mean(axis=0)).flatten()
        
        return composite_vector

    def analyze(self, user_text, recommendation_list, top_n=10):
        # Tentukan Target (Rata-rata skill dari Rekomendasi)
        target_vec = self.get_target_vector_from_recs(recommendation_list)
        if target_vec is None: 
            return []
        
        # Tentukan Posisi User (Skill yang sudah dimiliki)
        user_vec = self.vectorizer.transform([user_text]).toarray().flatten()
        
        # Hitung Gap Murni (Target - User)
        # Jika hasilnya Positif (+): Artinya Skill itu dibutuhkan di rekomendasi tapi user lemah/tidak punya.
        # Jika hasilnya Negatif (-): Artinya User sudah jago di skill itu (surplus).
        gap_vec = np.maximum(0, target_vec - user_vec)
        
        # Filter: Hanya ambil gap yang nilainya signifikan (> 0.01)
        gap_indices = np.where(gap_vec > 0.01)[0]
        
        if len(gap_indices) == 0:
            return []

        # Susun Hasil
        candidates = []
        for idx in gap_indices:
            skill_name = self.feature_names[idx]
            gap_score = gap_vec[idx] # Semakin besar skor, semakin urgent skill ini
            
            candidates.append({
                'skill': skill_name,
                'gap_score': gap_score
            })
            
        # Urutkan berdasarkan Gap Terbesar
        # Tidak perlu rumus rumit, cukup urutkan dari gap paling lebar
        df_gap = pd.DataFrame(candidates)
        results = df_gap.sort_values('gap_score', ascending=False).head(top_n)
        
        return results.to_dict('records')

# Inisiasi Modul
direct_gap_analyzer = DirectSkillGapModule(tfidf)

Contoh Penggunaan

In [48]:
# Ambil Input User
user_persona = persona_9_sales_marketing[0]
print(f"User Input: {user_persona[:100]}...")

# Cari Rekomendasi Pekerjaan dulu (Pakai fungsi search kamu)
rekomendasi = get_top_roles_by_text(user_persona, k=5)

# Cari Gap Skill
gaps = direct_gap_analyzer.analyze(user_persona, rekomendasi)

print(f"User Persona: {user_persona}")
print("\n--- REKOMENDASI PEKERJAAN ---")
for job in rekomendasi:
    print(f"- {job['job_title']}")

print("\n--- SKILL GAP (PRIORITAS UNTUK DIPELAJARI) ---")
if not gaps:
    print("Tidak ada gap skill signifikan.")
else:
    for i, item in enumerate(gaps, 1):
        # Score menggambarkan seberapa sering skill ini muncul di rekomendasi
        # tapi hilang di user.
        print(f"{i}. {item['skill']} (Gap Score: {item['gap_score']:.4f})")

User Input: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pe...


Batches: 100%|██████████| 1/1 [00:00<00:00, 25.77it/s]

User Persona: 
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.


--- REKOMENDASI PEKERJAAN ---
- AREA SALES MANAGER, KALIMANTAN
- AREA SALES MANAGER, KALIMANTAN
- Sales Marketing
- Marketing Cosmetics & SkinCare Manager
- Marketing & Development Staff

--- SKILL GAP (PRIORITAS UNTUK DIPELAJARI) ---
1. survey bahasa (Gap Score: 0.2538)
2. sales survey (Gap Score: 0.2310)
3. survey (Gap Score: 0.155